In [1]:
!python part3.py

============================ CN ============================
{'B-neutral': 1, 'I-neutral': 2, 'O': 3, 'B-positive': 4, 'I-positive': 5, 'B-negative': 6, 'I-negative': 7, 'stop': 8, 'start': 0}
[0.         0.32062739 0.25447525 0.12728138 0.26457619 0.19060167
 0.46363636 0.31932773 0.        ]
0  out of  546  tweets have been predicted.
100  out of  546  tweets have been predicted.
200  out of  546  tweets have been predicted.
300  out of  546  tweets have been predicted.
400  out of  546  tweets have been predicted.
500  out of  546  tweets have been predicted.
time to run the Viterbi algorithm for CN :  0.9942994117736816
============================ EN ============================
{'O': 1, 'B-INTJ': 2, 'B-PP': 3, 'B-NP': 4, 'I-NP': 5, 'B-VP': 6, 'B-PRT': 7, 'I-VP': 8, 'B-ADJP': 9, 'B-SBAR': 10, 'B-ADVP': 11, 'I-INTJ': 12, 'B-CONJP': 13, 'I-CONJP': 14, 'I-ADVP': 15, 'I-ADJP': 16, 'I-SBAR': 17, 'I-PP': 18, 'stop': 19, 'start': 0}
[0.         0.25878594 0.45525292 0.06424581 0.28684971

In [1]:
# coding: utf-8

from part2 import * 
from part3_fun import *
from check_fun import *
from preprocess import *
from collections import defaultdict
import numpy as np
import time



    

# main code block
if __name__ == '__main__':
    outfile = '/dev.p3.out'
    
#     languages = ['FR']
    # loop over languages
    for lang in languages:
        
        
        print ('============================',lang , '============================')
        # ====================================== training ======================================
        # reading tweets for particular language
        ptrain = data_from_file(lang + '/train') # unmodified
        train = mod_train (ptrain) # modified w/ start and stop states

        # getting sentiments and associated indices (w/ start and stop)
        sents = get_tags(ptrain) 
        print (sents)

        Y = get_counts(train)[0] # dictionary of sentiments and their counts
        diff_words = get_words(train)[0] # array of unique words 
        word_dict = get_words(train)[1] # dictionary of unique words and indices

        # emission and transmission parameter matrices
        emission_dict = get_emission2(train, 3) # dictionary with keys as (x, y) and values as emission probabilities
        em_mat = em_matrix(emission_dict, diff_words, sents) # emission matrix
        trans_mat = transition_params(train, Y, sents) # transition matrix
        # ======================================================================================
                
        # ========================================= validation set =========================================
        # A list of list of tuples of size 1. Each list in test is a tweet. 
        ptest = data_from_file(lang + '/dev.in')
        # test is a list of list. Each sublist is an array of words, 1 tweet
        ptest = [[word[0] for word in line] for line in ptest]
        test = mod_test(ptest) # modified with start and stop words
        # ==================================================================================================
        
        start = time.time()
        # ============================================ getting predictions ============================================
        # initializing list of optimal sentiment lists corresponding to each tweet 
        optimal_sentiments = []
        
        # loop that runs over all tweets for a given language to predict optimal sentiments
        for tweet in range(len(test)):
            
            # running Viterbi algorithm
            base_scores = np.ones([len(sents.keys()),1]) # initializing base case scores
            opt_ind_list = viterbi_algo (em_mat, trans_mat, word_dict, test[tweet], base_scores, 1, [], sents['O']) 
            
            # generating list of optimal sentiments for a given sentence
            inv_sents = dict (zip(sents.values(), sents.keys())) # swapping keys and values
            opt_sents = [inv_sents[opt_ind_list[i]] for i in range(len(opt_ind_list))]

            optimal_sentiments.append(opt_sents) # populating parent optimal sentiment list
            
            # printing iteration checks
            if (tweet % 100 == 0):
                print (tweet, ' out of ', len(test), ' tweets have been predicted.')
        
        predictions = []
        for tweet in range(len(optimal_sentiments)):
            predictions.append([(ptest[tweet][i], optimal_sentiments[tweet][i]) for i in range(len(optimal_sentiments[tweet]))])
        
        write_predictions(predictions, lang, outfile) # writing predictions to outfile
        # =============================================================================================================
        end = time.time()
        print('time to run the Viterbi algorithm for', lang, ': ', end - start)
    
    print ('============================ Predictions Complete ============================')
    
    





============================ CN ============================
{'B-neutral': 1, 'I-neutral': 2, 'O': 3, 'B-positive': 4, 'I-positive': 5, 'B-negative': 6, 'I-negative': 7, 'stop': 8, 'start': 0}
There are 22052 columns out of  22052 with probabilities > 0
0  out of  546  tweets have been predicted.
100  out of  546  tweets have been predicted.
200  out of  546  tweets have been predicted.
300  out of  546  tweets have been predicted.
400  out of  546  tweets have been predicted.
500  out of  546  tweets have been predicted.
time to run the Viterbi algorithm for CN :  2.443474531173706
============================ EN ============================
{'O': 1, 'B-INTJ': 2, 'B-PP': 3, 'B-NP': 4, 'I-NP': 5, 'B-VP': 6, 'B-PRT': 7, 'I-VP': 8, 'B-ADJP': 9, 'B-SBAR': 10, 'B-ADVP': 11, 'I-INTJ': 12, 'B-CONJP': 13, 'I-CONJP': 14, 'I-ADVP': 15, 'I-ADJP': 16, 'I-SBAR': 17, 'I-PP': 18, 'stop': 19, 'start': 0}
There are 3654 columns out of  3654 with probabilities > 0
0  out of  78  tweets have been predic